In [2]:
import os
import shutil
import pandas as pd

In [2]:
# dataDicts = []
# for root, dirs, files in os.walk('oasis3downloads'):
#     for file in files:
#         if 'nii.gz' in file:
#             filepath = os.path.join(root,file)
#             if '_MR_' in filepath:
#                 type = str(filepath).split('_')[-1]
#                 type = type.split('.nii.gz')[0]
#                 baseFolder = str(filepath).split(os.sep)[1]
#                 newPath = os.path.join('NIFTI',baseFolder,file)
#                 temp = pd.Series()
#                 temp = {'origpath':filepath, 'type':type, 'newpath':newPath}
#                 dataDicts.append(temp)
#             break

In [3]:
df = pd.read_csv('trainLabels.txt',names=['ID','label','extra'],sep='\t',low_memory=False,dtype={'ID':str,'label':int,'extra':int})
df

,type
NIFTI_slice_name,
sub-OAS30333_ses-d141_acq-PIB_pet__s6.nii.gz,pet
sub-OAS30333_ses-d141_acq-PIB_pet__s9.nii.gz,pet
sub-OAS30333_ses-d141_acq-PIB_pet__s22.nii.gz,pet
sub-OAS30333_ses-d141_acq-PIB_pet__s4.nii.gz,pet
sub-OAS30333_ses-d141_acq-PIB_pet__s7.nii.gz,pet
...,...
sub-OAS30603_sess-d2253_CT__s1.nii.gz,CT
sub-OAS30603_sess-d2253_CT__s3.nii.gz,CT
sub-OAS30603_sess-d2253_CT__s4.nii.gz,CT


In [4]:
print(df['type'].unique())

['pet' 'T2star' 'fieldmap' 'bold' 'CT' 'T2w' 'dwi' 'T1w' 'FLAIR' 'asl'
 'GRE' 'minIP' 'swi' 'angio' 'pasl' 'e2' nan]


In [9]:
df['description']=df['type']

In [12]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"FS":4,"PD":5,
             "SWI":6, "GRE":7, "T2*":8, "DWI":9, "ADC":10, "BOLD":11,
             "FlowSensitve":12,"PWI":13,"ASL":14,"Hippo":15,
             "DTI":16,"Other":17}
def assignLabel(dataframe):
    description = str(dataframe['description'])
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    otherCategories = ['fieldmap','scout','calibration','phase','localizer','nan','NotMri','pet','CT','e2']
    if description in otherCategories:
        return 17
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 5
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='FS':
        return 4
    if description =='pd':
        return 5
    if description =='swi' or description=='minIP':
        return 6
    if description.lower() =='gre': #from oasis3 can be magnitude or phase but no more description
        return 7
    if description =='T2star':
        return 8
    if description =='dwi':
        return 9
    if description =='adc':
        return 10
    if description =='bold':
        return 11
    if description =='angio': #flowsensitive
        return 12
    if description =='pwi' or description=='cbf':
        return 13
    if 'asl' in description:
        return 14
    if description == 'hippo':
        return 15
    if description == 'dti':
        return 16
    return -1

In [13]:
df['label'] = df.apply(assignLabel,axis=1)

In [15]:
df['label'].value_counts()

label
17    100625
11     67150
2      55725
0      54000
9      44825
6      36600
8      31725
14     20800
7      19450
3      16900
12     12175
Name: count, dtype: int64

In [ ]:
df.to_csv('Oasis3_trainLabels_extensive2.txt',index=False, header=False)